In [55]:
import json # library to handle JSON files

!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import pandas as pd

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


print('Libraries imported.')

Libraries imported.


Remove last row as it is not relevant to data

In [56]:
population_df = pd.DataFrame()

#There are 16 pages in the table
for x in range(17):
    pop_url = 'http://zipatlas.com/us/ny/zip-code-comparison/median-household-income.' + str(x+1) + '.htm'
    new_df = pd.read_html(pop_url)[10]
    new_df = new_df.rename(columns=new_df.iloc[0]).drop(new_df.index[0])
    population_df = pd.concat([population_df, new_df], axis=0)


population_df[['Latitude','Longitude']] = population_df.Location.str.split(",",expand=True)
del population_df['Location']
population_df

boroughs = ['New York, New York', 'Brooklyn, New York', 'Queens, New York', 'Bronx, New York', 'Staten Island, New York']

population_df = population_df.loc[population_df['City'].isin(boroughs)]
population_df.reset_index(drop=True, inplace=True)
population_df = population_df.rename(columns={'Avg. Income/H/hold': 'AvgIncome'})


# Remove rows with population of 0
population_df = population_df[population_df.Population != '0']
# Remove rows with 	Avg. Income of 0
population_df = population_df[population_df.AvgIncome != '$0.00']
population_df

,#,Zip Code,City,Population,AvgIncome,National Rank,Latitude,Longitude
0,25.,10007,"New York, New York",3522,"$112,947.00",#132,40.713973,-74.008336
1,29.,10280,"New York, New York",6614,"$108,536.00",#157,40.708501,-74.016828
2,30.,10162,"New York, New York",1726,"$108,416.00",#158,40.769298,-73.949942
3,34.,10004,"New York, New York",1225,"$101,868.00",#238,40.691870,-74.014109
4,57.,10069,"New York, New York",1403,"$88,091.00",#507,40.775551,-73.989689
...,...,...,...,...,...,...,...,...
114,1567.,10457,"Bronx, New York",69048,"$19,233.00","#30,729",40.846687,-73.898374
115,1578.,10456,"Bronx, New York",76656,"$16,664.00","#31,194",40.830877,-73.907954
116,1568.,11206,"Brooklyn, New York",69032,"$18,661.00","#30,855",40.702037,-73.942444
117,1579.,10474,"Bronx, New York",11354,"$16,339.00","#31,233",40.810203,-73.884437


In [57]:
def getNearbyFoodVenues(names, latitudes, longitudes, radius=500, section = 'food'):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&section={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            section, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Zip', 
                  'Zip Latitude', 
                  'Zip Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


CLIENT_ID = '5R0IEDEC5CE051431E0UXETF4STAPCSPMIAXV121CUIUBF0Z' # your Foursquare ID
CLIENT_SECRET = 'WVV4D4WTSEBNZFWBDBA3KESNPUGOFEKJB0IPPII1RSF4WYXW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100

ny_food = getNearbyFoodVenues(names=population_df['Zip Code'],
                                   latitudes=population_df['Latitude'],
                                   longitudes=population_df['Longitude']
                                  )

# Only focus on restarants
ny_food = ny_food[ny_food['Venue Category'].str.contains('Restaurant')]

# Remove Fast Food Places
ny_food = ny_food[ny_food['Venue Category'] != 'Fast Food Restaurant']
ny_food

10007
10280
10162
10004
10069
10282
10006
10022
10005
10024
10028
10021
10023
10128
10017
10312
10014
10016
10309
10307
10010
10011
10308
10003
10314
10012
10464
10471
11201
10019
10306
11215
11234
10044
10025
11217
10305
10018
10310
10465
10301
11231
11228
11209
11210
10303
10302
11236
10469
10304
10036
10001
10475
10463
10009
10461
11238
10470
10013
11229
11203
10466
11218
11214
10462
11222
11230
11223
11204
10033
10038
11235
11225
11220
11226
10034
10467
11232
11205
10040
10473
11208
10037
10039
10468
11219
11213
11224
10032
11216
10031
10029
10030
11207
10002
11211
11212
10472
10027
10026
11237
11239
11233
11221
10458
10460
10459
10453
10452
10454
10451
10455
10457
10456
11206
10474
10035


,Zip,Zip Latitude,Zip Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
2,10007,40.713973,-74.008336,Nish Nūsh,40.715537,-74.007725,Falafel Restaurant
3,10007,40.713973,-74.008336,Little Park,40.715487,-74.009133,American Restaurant
5,10007,40.713973,-74.008336,Racines,40.714754,-74.007581,French Restaurant
6,10007,40.713973,-74.008336,The Hummus & Pita Co,40.714453,-74.006853,Falafel Restaurant
8,10007,40.713973,-74.008336,Sophie's Cuban Cuisine,40.714803,-74.007656,Cuban Restaurant
...,...,...,...,...,...,...,...
4662,10035,40.795487,-73.929501,Bistro Casa Azul,40.796607,-73.932342,Mexican Restaurant
4670,10035,40.795487,-73.929501,Wing Wah Chinese Restaurant,40.797486,-73.933994,Chinese Restaurant
4674,10035,40.795487,-73.929501,La Cabana,40.796770,-73.934769,Mexican Restaurant
4675,10035,40.795487,-73.929501,Teng Dragon,40.796110,-73.935021,Chinese Restaurant


Postcode 10301 returned no results from Foursquare so we will remove it from our main dataset population_df

In [ ]:
ny_food = ny_food.dropna(how='any',axis=0)

print(ny_food.loc[ny_food['Zip'] == '10301'])
indexNames = population_df[population_df['Zip Code'] == '10301'].index
population_df.drop(indexNames , inplace=True)
population_df

,#,Zip Code,City,Population,Avg. Income/H/hold,National Rank,Latitude,Longitude


In [67]:
zip_food_count = ny_food.groupby('Zip').count()
zip_food_count.reset_index(inplace=True)


merged_df = pd.merge(left=population_df, right=zip_food_count[['Zip','Venue']], how='left', left_on='Zip Code', right_on='Zip')
merged_df.reset_index(drop=True, inplace=True)  
merged_df_clean = merged_df.drop(['#', 'City', 'National Rank', 'Latitude','Longitude', 'Zip'], axis=1)


merged_df_clean['AvgIncome'] = merged_df_clean['AvgIncome'].str.replace('$', '')
merged_df_clean['AvgIncome'] = merged_df_clean['AvgIncome'].str.replace(',', '')
merged_df_clean['AvgIncome'] = merged_df_clean['AvgIncome'].astype(float) 
merged_df_clean['Population'] = merged_df_clean['Population'].astype(float) 

merged_df_clean

,Zip Code,Population,AvgIncome,Venue
0,10007,3522.0,112947.0,31.0
1,10280,6614.0,108536.0,28.0
2,10162,1726.0,108416.0,27.0
3,10004,1225.0,101868.0,4.0
4,10069,1403.0,88091.0,3.0
...,...,...,...,...
111,10457,69048.0,19233.0,5.0
112,10456,76656.0,16664.0,5.0
113,11206,69032.0,18661.0,15.0
114,10474,11354.0,16339.0,3.0


In [68]:
list(merged_df_clustering.isnull()['Venue'])

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False]

In [60]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 10

merged_df_clustering = merged_df_clean.drop('Zip Code', 1)
merged_df_clustering
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(merged_df_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]


merged_df_clean.insert(0, 'Cluster Labels', kmeans.labels_)



#toronto_df.rename(columns={"Neighbourhood": "Neighborhood"}, inplace=True)
#tor_merged = toronto_df

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
#tor_merged = tor_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

#tor_merged.head() # check the last columns!

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [24]:
# High Population, medium Income
merged_df_clean[merged_df_clean['Cluster Labels'] == 0]

,Cluster Labels,Zip Code,Population,AvgIncome,Venue
42,0,11209,69840.0,44518.0,100
43,0,11210,62308.0,42967.0,30
47,0,10469,63345.0,42102.0,13
52,0,10463,65576.0,40497.0,27
53,0,10009,58595.0,40176.0,84
60,0,10466,68787.0,37141.0,9
63,0,10462,72077.0,33735.0,11
68,0,10033,58259.0,31348.0,90
71,0,11225,63774.0,30192.0,45
79,0,10473,56166.0,27733.0,4


In [25]:
# Low Population, medium Income
merged_df_clean[merged_df_clean['Cluster Labels'] == 1]

,Cluster Labels,Zip Code,Population,AvgIncome,Venue
26,1,10464,4634.0,57458.0,1
33,1,10044,9520.0,49976.0,13
37,1,10018,4255.0,48705.0,100
38,1,10310,22852.0,46198.0,14
44,1,10303,23530.0,42463.0,2
45,1,10302,16406.0,42452.0,10
49,1,10036,18751.0,41002.0,100
50,1,10001,17310.0,40932.0,69
56,1,10470,15780.0,38464.0,28
57,1,10013,25042.0,38304.0,100


In [26]:
# Low Population, Highest Income
merged_df_clean[merged_df_clean['Cluster Labels'] == 2]

,Cluster Labels,Zip Code,Population,AvgIncome,Venue
0,2,10007,3522.0,112947.0,83
1,2,10280,6614.0,108536.0,76
2,2,10162,1726.0,108416.0,56
3,2,10004,1225.0,101868.0,11
4,2,10069,1403.0,88091.0,16
5,2,10282,1574.0,86066.0,18
6,2,10006,1447.0,81334.0,56
8,2,10005,884.0,79517.0,92


In [28]:
# High Population, High Income
merged_df_clean[merged_df_clean['Cluster Labels'] == 3]

,Cluster Labels,Zip Code,Population,AvgIncome,Venue
9,3,10024,61414.0,78066.0,54
10,3,10028,44987.0,77565.0,74
12,3,10023,62206.0,72424.0,99
13,3,10128,59856.0,70031.0,100
15,3,10312,58867.0,67728.0,2
17,3,10016,51217.0,66342.0,100
21,3,10011,46669.0,61986.0,79
23,3,10003,53673.0,60891.0,91
28,3,11201,47746.0,56293.0,28
30,3,10306,55698.0,55413.0,4


In [27]:
# Highest Population, High Income
merged_df_clean[merged_df_clean['Cluster Labels'] == 4]

,Cluster Labels,Zip Code,Population,AvgIncome,Venue
11,4,10021,102078.0,75472.0,72
24,4,10314,84821.0,59560.0,3
32,4,11234,86898.0,51446.0,2
34,4,10025,97086.0,49733.0,49
46,4,11236,96301.0,42370.0,11


In [29]:
# High Population, Low Income
merged_df_clean[merged_df_clean['Cluster Labels'] == 5]

,Cluster Labels,Zip Code,Population,AvgIncome,Venue
58,5,11229,80554.0,37812.0,38
59,5,11203,84498.0,37341.0,22
61,5,11218,74611.0,36432.0,31
62,5,11214,83546.0,33765.0,47
65,5,11230,88933.0,32327.0,30
66,5,11223,77691.0,32104.0,27
67,5,11204,74880.0,31798.0,42
70,5,11235,77451.0,31013.0,38
72,5,11220,92718.0,30152.0,47
73,5,11226,106154.0,29498.0,22


In [30]:
# Medium Population, Low Income
merged_df_clean[merged_df_clean['Cluster Labels'] == 6]

,Cluster Labels,Zip Code,Population,AvgIncome,Venue
74,6,10034,41756.0,29479.0,42
77,6,11205,35622.0,28070.0,48
78,6,10040,46599.0,27905.0,32
86,6,11224,51205.0,21281.0,15
98,6,10026,30377.0,22491.0,50
99,6,11237,48910.0,23104.0,53
104,6,10460,53707.0,19517.0,15
105,6,10459,39080.0,17498.0,11
108,6,10454,34976.0,14271.0,30
109,6,10451,40961.0,20307.0,28


In [31]:
merged_df_clean[merged_df_clean['Cluster Labels'] == 7]

,Cluster Labels,Zip Code,Population,AvgIncome,Venue
35,7,11217,35353.0,49567.0,44
36,7,10305,38450.0,48944.0,10
39,7,10465,42500.0,45650.0,25
40,7,11231,32974.0,45154.0,17
41,7,11228,41172.0,44932.0,12
48,7,10304,38972.0,41041.0,2
51,7,10475,38086.0,40528.0,6
54,7,10461,49808.0,40024.0,15
55,7,11238,48965.0,39917.0,48
64,7,11222,39360.0,33578.0,40


In [32]:
merged_df_clean[merged_df_clean['Cluster Labels'] == 8]

,Cluster Labels,Zip Code,Population,AvgIncome,Venue
69,8,10038,15574.0,31316.0,87
76,8,11232,27723.0,28395.0,57
81,8,10037,16984.0,26561.0,13
82,8,10039,21737.0,17370.0,23
91,8,10030,25847.0,17970.0,67
100,8,11239,14620.0,16919.0,5
114,8,10474,11354.0,16339.0,11


In [33]:
merged_df_clean[merged_df_clean['Cluster Labels'] == 9]

,Cluster Labels,Zip Code,Population,AvgIncome,Venue
7,9,10022,30642.0,80406.0,100
14,9,10017,16201.0,69273.0,100
16,9,10014,32667.0,66601.0,100
18,9,10309,27207.0,65397.0,1
19,9,10307,11669.0,64159.0,12
20,9,10010,26408.0,62467.0,83
22,9,10308,26451.0,61868.0,17
25,9,10012,26000.0,58313.0,100
27,9,10471,23413.0,56488.0,1
29,9,10019,36012.0,55869.0,100


In [51]:
ny_food
#ny_food['Venue Category'].unique()


